In [20]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import numpy as np
import re

from tensorflow.keras.layers import Dense, LSTM, Input, Dropout, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [21]:
with open('train_data_true.txt', 'r', encoding='utf-8') as f:
    texts_true = f.readlines()
    texts_true[0] = texts_true[0].replace('\ufeff', '') #убираем первый невидимый символ

with open('train_data_false.txt', 'r', encoding='utf-8') as f:
    texts_false = f.readlines()
    texts_false[0] = texts_false[0].replace('\ufeff', '') #убираем первый невидимый символ

In [22]:
texts = texts_true + texts_false
count_true = len(texts_true)
count_false = len(texts_false)
total_lines = count_true + count_false
print(count_true, count_false, total_lines)

84 88 172


In [23]:
maxWordsCount = 1000
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»', lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts(texts)

In [24]:
dist = list(tokenizer.word_counts.items())
print(dist[:10])
print(texts[0][:100])

[('думайте', 1), ('позитивно', 4), ('и', 50), ('верьте', 3), ('в', 38), ('свою', 4), ('способность', 1), ('достигать', 1), ('отличных', 1), ('результатов', 1)]
Думайте позитивно и верьте в свою способность достигать отличных результатов. 



In [25]:
max_text_len = 10
data = tokenizer.texts_to_sequences(texts)
data_pad = pad_sequences(data, maxlen=max_text_len)
print(data_pad)

[[197  54   2 ... 199 200 201]
 [  0   4 202 ... 205   3  67]
 [206   3  67 ...   4 208 209]
 ...
 [  0  20  62 ...  53 850 851]
 [  0   0  43 ...  33   1 853]
 [  0   0   0 ...  70  65 194]]


In [26]:
print( list(tokenizer.word_index.items()) )

[('не', 1), ('и', 2), ('в', 3), ('вы', 4), ('на', 5), ('я', 6), ('а', 7), ('только', 8), ('что', 9), ('все', 10), ('это', 11), ('если', 12), ('жизнь', 13), ('просто', 14), ('с', 15), ('чем', 16), ('никогда', 17), ('чтобы', 18), ('к', 19), ('у', 20), ('больше', 21), ('себя', 22), ('то', 23), ('всегда', 24), ('вас', 25), ('меня', 26), ('за', 27), ('вам', 28), ('когда', 29), ('от', 30), ('мне', 31), ('быть', 32), ('жизни', 33), ('есть', 34), ('но', 35), ('сделать', 36), ('без', 37), ('позитивным', 38), ('никому', 39), ('как', 40), ('ты', 41), ('тем', 42), ('для', 43), ('день', 44), ('позитивное', 45), ('своих', 46), ('мы', 47), ('много', 48), ('тебя', 49), ('никто', 50), ('такой', 51), ('деньги', 52), ('денег', 53), ('позитивно', 54), ('свою', 55), ('которые', 56), ('значит', 57), ('так', 58), ('них', 59), ('мечты', 60), ('нужно', 61), ('того', 62), ('ничего', 63), ('лучше', 64), ('надо', 65), ('верьте', 66), ('понедельник', 67), ('живи', 68), ('позволяйте', 69), ('хорошее', 70), ('можете

In [27]:
X = data_pad
Y = np.array([[1, 0]]*count_true + [[0, 1]]*count_false)
print(X.shape, Y.shape)

indeces = np.random.choice(X.shape[0], size=X.shape[0], replace=False)
X = X[indeces]
Y = Y[indeces]

(172, 10) (172, 2)


In [28]:
model = Sequential()
model.add(Embedding(maxWordsCount, 128, input_length = max_text_len))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(64))
model.add(Dense(2, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 10, 128)           128000    
                                                                 
 lstm (LSTM)                 (None, 10, 128)           131584    
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dense_1 (Dense)             (None, 2)                 130       
                                                                 
Total params: 309,122
Trainable params: 309,122
Non-trainable params: 0
_________________________________________________________________


In [29]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=Adam(0.0001))

history = model.fit(X, Y, batch_size=32, epochs=50)

reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))

Epoch 1/50
6/6 [==============================] - 6s 34ms/step - loss: 0.6919 - accuracy: 0.5930
Epoch 2/50
6/6 [==============================] - 0s 33ms/step - loss: 0.6886 - accuracy: 0.6453
Epoch 3/50
6/6 [==============================] - 0s 32ms/step - loss: 0.6855 - accuracy: 0.6512
Epoch 4/50
6/6 [==============================] - 0s 32ms/step - loss: 0.6816 - accuracy: 0.6395
Epoch 5/50
6/6 [==============================] - 0s 32ms/step - loss: 0.6765 - accuracy: 0.6686
Epoch 6/50
6/6 [==============================] - 0s 31ms/step - loss: 0.6703 - accuracy: 0.6860
Epoch 7/50
6/6 [==============================] - 0s 32ms/step - loss: 0.6629 - accuracy: 0.7093
Epoch 8/50
6/6 [==============================] - 0s 32ms/step - loss: 0.6532 - accuracy: 0.7558
Epoch 9/50
6/6 [==============================] - 0s 31ms/step - loss: 0.6398 - accuracy: 0.8140
Epoch 10/50
6/6 [==============================] - 0s 35ms/step - loss: 0.6211 - accuracy: 0.8430
Epoch 11/50
6/6 [============

In [31]:
def sequence_to_text(list_of_indices):
    words = [reverse_word_map.get(letter) for letter in list_of_indices]
    return(words)

mas = ["Негативное", "Позитивное"]

t = "Я люблю позитивное настроение".lower()
data = tokenizer.texts_to_sequences([t])
data_pad = pad_sequences(data, maxlen=max_text_len)
print( sequence_to_text(data[0]) )

res = model.predict(data_pad)
print(mas[np.argmax(res)])

['я', 'позитивное', 'настроение']
1/1 [==============================] - 0s 27ms/step
Позитивное
